# SimCSE with Dummy Encoder (Offline Version)
This notebook trains a simple embedding model (no Hugging Face, works offline).

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import random

# Dummy dataset (toy obituary sentences)
sentences = [
    'John was a kind man who loved his family.',
    'Mary will be remembered for her generosity.',
    'David was a devoted husband and father.',
    'Sarah touched many lives with her kindness.',
    'Michael was a teacher and a mentor to many.'
]

class TextDataset(Dataset):
    def __init__(self, texts):
        self.texts = texts
        self.vocab = {w:i for i,w in enumerate(set(' '.join(texts).split()))}
    def __len__(self): return len(self.texts)
    def __getitem__(self, idx):
        words = self.texts[idx].split()
        ids = [self.vocab[w] for w in words]
        return torch.tensor(ids)

dataset = TextDataset(sentences)
dataloader = DataLoader(dataset, batch_size=2, shuffle=True)

class DummyEncoder(nn.Module):
    def __init__(self, vocab_size, hidden_dim=64):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, hidden_dim)
    def forward(self, x):
        return self.embedding(x).mean(dim=1)

encoder = DummyEncoder(len(dataset.vocab))
optimizer = optim.Adam(encoder.parameters(), lr=1e-3)
criterion = nn.CosineEmbeddingLoss()

# Training loop (toy contrastive learning)
for epoch in range(5):
    for batch in dataloader:
        if len(batch) < 2: continue
        x1, x2 = batch[0], batch[1]
        emb1 = encoder(x1.unsqueeze(0))
        emb2 = encoder(x2.unsqueeze(0))
        label = torch.tensor([1.0])  # assume positive pair
        loss = criterion(emb1, emb2, label)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    print(f'Epoch {epoch+1}, Loss: {loss.item():.4f}')

# Save model
torch.save(encoder.state_dict(), 'dummy_encoder.pt')
print('Model saved as dummy_encoder.pt')